In [5]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = None #'mypassword'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

Connected to Redis


True

# Personalized Product Recommendations
## Description
This use case demonstrates how to provide personalized product recommendations to users based on their past interactions and preferences.

## Redis Data Structures Used
- JSON
- Search

In [7]:
r.flushdb()
# Mock data for personalized product recommendations
mock_data = {
    'user:1001': {'preferences': ['loan', 'cards'], 'recent_views': ['product:2001', 'product:2002']},
    'product:2001': {'category': 'loan', 'name': 'Home Loan', 'base_rate': 7.5},
    'product:2002': {'category': 'cards', 'name': 'Gold Card', 'eligibility': 'salary > 1 Lakh pm'},
    'product:2003': {'category': 'savings', 'name': 'Savings Account', 'base_rate': 2.5},
    'product:2004': {'category': 'loan', 'name': 'Personal Loan', 'base_rate':10.5 },
    'product:3001': {'category': 'savings', 'name': 'Fixed Deposit (short)', 'base_rate': 4.5},
    'product:3002': {'category': 'savings', 'name': 'Fixed Deposit (long)', 'base_rate': 6.5},
    'product:3003': {'category': 'cards', 'name': 'Silver Card', 'eligibility': 'salary > 25000 pm'},
    'product:3004': {'category': 'loan', 'name': 'Overdraft Account', 'base_rate': 0}
}

# Ingest mock data into Redis
for key, value in mock_data.items():
    r.json().set(key, '$', json.dumps(value))

# Function to get personalized recommendations
def get_personalized_recommendations(user_id):
    user_data = json.loads(client.json().get(f'user:{user_id}', '$')[0])
    preferences = user_data['preferences']
    print(f"User ({user_id}) Category Preferences : {preferences}")
    recent_views = user_data['recent_views']
    print(f"User ({user_id}) Recent Views : {recent_views}")
    recommendations = []
    for pref in preferences:
        for key in client.scan_iter(match='product:*'):
            product = json.loads(client.json().get(key, '$')[0])
            if product['category'] == pref and key not in recent_views:
                recommendations.append(product)
    return recommendations

# Get recommendations for a user
user_id = 1001
recommendations = get_personalized_recommendations(user_id)
print(f'Recommendations for user {user_id}:')
for x in recommendations:print(x)

User (1001) Category Preferences : ['loan', 'cards']
User (1001) Recent Views : ['product:2001', 'product:2002']
Recommendations for user 1001:
{'category': 'loan', 'name': 'Personal Loan', 'base_rate': 10.5}
{'category': 'loan', 'name': 'Overdraft Account', 'base_rate': 0}
{'category': 'cards', 'name': 'Silver Card', 'eligibility': 'salary > 25000 pm'}
